<a href="https://colab.research.google.com/github/fafnerzhang/data-course-sample/blob/dev/rule_based_recommandation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-26 13:47:43--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.2’

All_Beauty.csv.2    100%[===================>]  14.78M  67.9MB/s    in 0.2s    

2021-12-26 13:47:43 (67.9 MB/s) - ‘All_Beauty.csv.2’ saved [15499476/15499476]

--2021-12-26 13:47:43--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.2’

meta_All_Beauty.jso 100%[===================>]   9.85M  57.6MB/s    in 0.2s   

In [ ]:
metadata = getDF('./meta_All_Beauty.json.gz')
ratings = pd.read_csv('./All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


# 資料整理

In [ ]:
metadata['ranking'] = metadata['rank'].str.split(expand = True)[0]
metadata['ranking'] = metadata['ranking'].str.replace(',', '')
metadata['ranking'] = pd.to_numeric(metadata['ranking'])

In [ ]:
metadata['sub_category'] = metadata['rank'].str.split('in ').str[1]
metadata['sub_category'] = metadata['sub_category'].str.replace(r'&amp;', '', regex=True)
metadata['sub_category'] = metadata['sub_category'].str.replace(r'\(', '', regex=True)
metadata['sub_category'] = metadata['sub_category'].str.replace(r'&', '', regex=True)
print(metadata['sub_category'].value_counts())
metadata = metadata.drop(metadata[(metadata['sub_category'] != 'Beauty  Personal Care ')].index)
metadata

Beauty  Personal Care        32380
Grocery  Gourmet Food           38
Health  Household               21
Sports  Outdoors                 3
Toys  Games                      3
Clothing, Shoes  Jewelry         2
Baby                             2
Tools  Home Improvement          1
Home  Kitchen                    1
Automotive                       1
Name: sub_category, dtype: int64


,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,ranking,sub_category
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[],2938573.0,Beauty Personal Care
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[],872854.0,Beauty Personal Care
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[],956696.0,Beauty Personal Care
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,1870258.0,Beauty Personal Care
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,67701.0,Beauty Personal Care
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32887,[],,[],,"Barielle Pro Textured Grip Cuticle Nipper, Purple",[],,,[],"2,145,325 in Beauty & Personal Care (",[],{'ASIN: ': 'B01HIWLLUK'},All Beauty,,,$9.95,B01HIWLLUK,[],[],2145325.0,Beauty Personal Care
32888,[],,[],,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,[],,Salon Perfect,[],"1,639,713 in Beauty & Personal Care (",[],"{'ASIN: ': 'B01HJ1K3YK', 'UPC:': '671635851871'}",All Beauty,,,,B01HJ1K3YK,[],[],1639713.0,Beauty Personal Care
32889,[],,[],,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,"[B01KON9B4S, B079X3YFXS, B00M79OYS6, B000JN4CR...",,,[],"207,410 in Beauty & Personal Care (","[B01KON9B4S, B000JN4CR0, B071ZHMRHS, B01HJ84TN...","{'Shipping Weight:': '1 pounds (', 'ASIN: ': '...",All Beauty,,,$55.63,B01HJ84SGM,[],[],207410.0,Beauty Personal Care
32890,[],,[Brand new and high quality<br> Enables fast v...,,12 White Feather Shuttlecocks Birdies Badminto...,[],,GBSTORE,[],"965,673 in Beauty & Personal Care (",[],"{'Shipping Weight:': '4.8 ounces (', 'ASIN: ':...",All Beauty,,,$12.99,B01HJASD20,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,965673.0,Beauty Personal Care


In [ ]:
metadata['text_data'] = metadata.agg(lambda x: f'{x["title"]}, {" ".join(x["description"])}', axis=1)
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
text = metadata['text_data'].tolist()

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
from yellowbrick.cluster import KElbowVisualizer

In [ ]:
vectorizor = TfidfVectorizer(min_df=200)
for idx, value in enumerate(text):
    if not isinstance(value, str):
        text[idx] = ''
feature = vectorizor.fit_transform(text).toarray()
len(vectorizor.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


831

In [ ]:
model = AgglomerativeClustering(n_clusters=20)
result = model.fit_predict(feature)

In [ ]:
metadata['specify'] = pd.Series(result)

In [ ]:
metadata['specify'].value_counts()

9.0     12904
6.0      2962
18.0     2131
3.0      2098
0.0      1578
2.0      1353
15.0     1169
4.0       984
1.0       979
8.0       847
16.0      813
19.0      657
12.0      547
10.0      543
14.0      542
7.0       457
5.0       365
11.0      354
13.0      321
17.0      268
Name: specify, dtype: int64

In [ ]:
def transform(data):
  res = dict()
  for item, row in data.iterrows():
    res[row['asin']] = row['specify']
  return res
cluster_dict = transform(metadata)

In [ ]:
def construct(data):
  if cluster_dict.get(data):
    return cluster_dict[data]
  else:
    return 22.0
ratings['cluster'] = ratings['asin'].apply(construct)

In [ ]:
ratings['cluster'].value_counts()

9.0     147769
6.0      38222
18.0     34303
22.0     25605
3.0      19745
10.0     15038
2.0      12501
15.0     10459
1.0       7506
16.0      7414
19.0      7219
14.0      6686
12.0      6624
4.0       6424
8.0       5791
5.0       4449
13.0      3683
7.0       3333
11.0      2959
17.0      2319
Name: cluster, dtype: int64

In [ ]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [ ]:
def recommender(training_data, group, time, users=[], k=10):
  if group > 19:
    group = 22
  target = training_data[(training_data['cluster'] == group)&
                (training_data['DATE']>=time)&
                (training_data['overall']>=training_data['overall'].mean())]
  brand_2 = target['asin'].value_counts().sort_values(ascending=False).keys()[:k]
  recommendations = {user: brand_2.tolist() for user in users}
  return recommendations

def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
  '''
  * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
  * ratings_by_user: dict 利用訓練資料學習的推薦商品
  * method: str
  * score: float
  '''
  total = 0
  for d in ratings_testings_by_user:
      if d in ratings_by_user:
          total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

  score = total / len(ratings_testings)
  return score

In [ ]:
time_list = ['2018-07-01','2018-05-01', '2018-01-01', '2016-01-01']
for time in time_list:
  print(f'time: {time}')
  for i in range(21):
    ratings_by_user = recommender(ratings_trainings, i, time, users)
    score = evaluate(ratings_testings_by_user, ratings_by_user)
    print(f'{i}, score: {score}')
  print('='*50)

time: 2018-07-01
0, score: 0.0
1, score: 0.005084745762711864
2, score: 0.03898305084745763
3, score: 0.01694915254237288
4, score: 0.0
5, score: 0.001694915254237288
6, score: 0.00847457627118644
7, score: 0.003389830508474576
8, score: 0.001694915254237288
9, score: 0.1423728813559322
10, score: 0.001694915254237288
11, score: 0.020338983050847456
12, score: 0.04576271186440678
13, score: 0.0
14, score: 0.00847457627118644
15, score: 0.04745762711864407
16, score: 0.01694915254237288
17, score: 0.001694915254237288
18, score: 0.03389830508474576
19, score: 0.0
20, score: 0.013559322033898305
time: 2018-05-01
0, score: 0.0
1, score: 0.001694915254237288
2, score: 0.03220338983050847
3, score: 0.01694915254237288
4, score: 0.0
5, score: 0.005084745762711864
6, score: 0.006779661016949152
7, score: 0.003389830508474576
8, score: 0.003389830508474576
9, score: 0.12542372881355932
10, score: 0.001694915254237288
11, score: 0.020338983050847456
12, score: 0.04406779661016949
13, score: 0.0

In [ ]:
metadata[metadata['specify']==9][:10]['title']

0         Loud 'N Clear&trade; Personal Sound Amplifier
1     No7 Lift &amp; Luminate Triple Action Serum 50...
2       No7 Stay Perfect Foundation Cool Vanilla by No7
3     Wella Koleston Perfect Hair Colour 44/44 Mediu...
4     Lacto Calamine Skin Balance Oil control 120 ml...
5     Mary Kay Satin Hands Hand Cream Travel MINI Si...
8                                            Mali princ
9     Moleskine Payne's Grey Card Wallet (Moleskine ...
10                              L'eternauta, il romanzo
11    Hello Kitty LALA Lovely Kids Girls Summer Slip...
Name: title, dtype: object